In [ ]:
import os
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import cv2
import PIL
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
import warnings
warnings.filterwarnings(action="ignore")


from keras import backend as K
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from google.colab import files
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#IMPORT KAGGLE DATASET  UPDATED

In [ ]:
# Mount Google Drive (Optional if saving data to Drive)
# from google.colab import drive
# drive.mount('/content/drive')

# Create the Kaggle directory and dynamically write the Kaggle JSON data
import os
kaggle_json_content = 'ENTER KAGGLE.JSON DATA HERE'

os.makedirs("/root/.kaggle", exist_ok=True)
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(kaggle_json_content)

# Set permissions for the kaggle.json file
!chmod 600 /root/.kaggle/kaggle.json

# Install Kaggle CLI
!pip install kaggle

# Download the dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Unzip the dataset
import zipfile
with zipfile.ZipFile("chest-xray-pneumonia.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")
print("Dataset extracted to /content/dataset")


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [01:45<00:00, 24.2MB/s]
100% 2.29G/2.29G [01:45<00:00, 23.3MB/s]
Dataset extracted to /content/dataset


#SET UP PATH FOR TRAINING,TESTING AND VALIDATION DIRECTORY.

In [ ]:
# Dataset Paths
healthy_dirs = [
    r'/content/dataset/chest_xray/test/NORMAL',
    r'/content/dataset/chest_xray/train/NORMAL',
    r'/content/dataset/chest_xray/val/NORMAL',
]

pneumonia_dirs = [
    r'/content/dataset/chest_xray/test/PNEUMONIA',
    r'/content/dataset/chest_xray/train/PNEUMONIA',
    r'/content/dataset/chest_xray/val/PNEUMONIA'
]

filepaths, labels = [], []
dict_lists = [healthy_dirs, pneumonia_dirs]
class_labels = ['Normal', 'Pneumonia']

for i, dir_list in enumerate(dict_lists):
    for j in dir_list:
        flist = os.listdir(j)
        for f in flist:
            filepaths.append(os.path.join(j, f))
            labels.append(class_labels[i])

pneumonia_df = pd.DataFrame({"filepaths": filepaths, "labels": labels})
print(pneumonia_df.head())
print(pneumonia_df["labels"].value_counts())

# Train, Test, Validation Splits
train_set, test_set = train_test_split(pneumonia_df, test_size=0.3, random_state=42)
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=42)

                                           filepaths  labels
0  /content/dataset/chest_xray/test/NORMAL/NORMAL...  Normal
1  /content/dataset/chest_xray/test/NORMAL/NORMAL...  Normal
2  /content/dataset/chest_xray/test/NORMAL/NORMAL...  Normal
3  /content/dataset/chest_xray/test/NORMAL/IM-002...  Normal
4  /content/dataset/chest_xray/test/NORMAL/NORMAL...  Normal
labels
Pneumonia    4273
Normal       1583
Name: count, dtype: int64


#DATA AUGMENTATION

In [ ]:
# Data Augmentation and Loading
image_gen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

train_gen = image_gen.flow_from_dataframe(
    dataframe=train_set, x_col="filepaths", y_col="labels",
    target_size=(244, 244), class_mode="categorical", batch_size=4, shuffle=True
)
val_gen = image_gen.flow_from_dataframe(
    dataframe=val_set, x_col="filepaths", y_col="labels",
    target_size=(244, 244), class_mode="categorical", batch_size=4, shuffle=False
)
test_gen = image_gen.flow_from_dataframe(
    dataframe=test_set, x_col="filepaths", y_col="labels",
    target_size=(244, 244), class_mode="categorical", batch_size=4, shuffle=False
)

classes = list(train_gen.class_indices.keys())
print("Classes:", classes)

Found 3279 validated image filenames belonging to 2 classes.
Found 820 validated image filenames belonging to 2 classes.
Found 1757 validated image filenames belonging to 2 classes.
Classes: ['Normal', 'Pneumonia']


#CNN MODEL

In [ ]:
from tensorflow.keras.layers import MaxPooling2D
# Define the Model
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)

class_count = len(list(train_gen.class_indices.keys()))

model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu", input_shape= img_shape),
    MaxPooling2D((2, 2)),

    Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu", input_shape= img_shape),
    MaxPooling2D((2, 2)),

    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),

    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),


    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(256,activation = "relu"),
    Dropout(0.3),
    Dense(128,activation = "relu"),
    Dropout(0.3),
    Dense(class_count, activation = "softmax")
])

#model.compile(Adamax(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 112, 112, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 56, 56, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 28, 28, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 28, 28, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 14, 14, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 14, 14, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       6,422,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 8,024,514 (30.61 MB)

 Trainable params: 8,024,514 (30.61 MB)

 Non-trainable params: 0 (0.00 B)

#MODEL TRAINING


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adamax(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])


# Training the Model
history = model.fit(train_gen, epochs=10, validation_data=val_gen, verbose=1)


Epoch 1/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 59s 58ms/step - accuracy: 0.8262 - loss: 0.3822 - val_accuracy: 0.9354 - val_loss: 0.1782
Epoch 2/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.9417 - loss: 0.1552 - val_accuracy: 0.9488 - val_loss: 0.1444
Epoch 3/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 44s 54ms/step - accuracy: 0.9506 - loss: 0.1276 - val_accuracy: 0.9585 - val_loss: 0.1295
Epoch 4/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 43s 52ms/step - accuracy: 0.9616 - loss: 0.1125 - val_accuracy: 0.9585 - val_loss: 0.1517
Epoch 5/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 44s 53ms/step - accuracy: 0.9695 - loss: 0.0818 - val_accuracy: 0.9549 - val_loss: 0.1198
Epoch 6/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 40s 49ms/step - accuracy: 0.9712 - loss: 0.0782 - val_accuracy: 0.9610 - val_loss: 0.1255
Epoch 7/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.9769 - loss: 0.0650 - val_accuracy: 0.9549 - val_loss: 0.1425
Epoch 8/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 41s 51ms/step - accuracy: 0.9797 - loss: 0.0585 - 

#MODEL TESTING

In [ ]:
# Evaluate the Model
test_loss, test_accuracy = model.evaluate(test_gen, verbose=1)
print(f"Test Accuracy: {test_accuracy:.2f}")

440/440 ━━━━━━━━━━━━━━━━━━━━ 18s 40ms/step - accuracy: 0.9623 - loss: 0.1726
Test Accuracy: 0.97


#SAVE THE MODEL

In [ ]:
# model.save("model.keras")  # Saves in recommended Keras format
# from tensorflow import keras

# model = keras.models.load_model("model.keras")



model.save("model.h5")
from tensorflow import keras

model = keras.models.load_model("model.h5")


#MODEL DEPLOYMENT

In [ ]:

# import gradio as gr
# import tensorflow as tf
# import numpy as np
# from PIL import Image

# # Load trained model
# model = tf.keras.models.load_model("model.h5")

# # Define class labels
# class_labels = {0: "Normal", 1: "Pneumonia"}

# # Preprocessing function (compatible with MobileNetV2)
# def preprocess_image(img):
#     img = img.resize((244, 244))  # Resize to match training size
#     img = np.array(img)  # Convert to numpy array
#     img = tf.keras.applications.mobilenet_v2.preprocess_input(img)  # Apply MobileNetV2 preprocessing
#     img = np.expand_dims(img, axis=0)  # Add batch dimension
#     return img

# # Prediction function
# def predict(img):
#     img = preprocess_image(img)
#     prediction = model.predict(img)[0]  # Get prediction
#     class_index = np.argmax(prediction)  # Get highest probability class
#     confidence = float(np.max(prediction))  # Get confidence score
#     return {class_labels[class_index]: confidence}

# # Example Images (Stored in the main directory)
# examples = [
#     ["normal_1.jpg"],
#     ["normal_2.jpg"],
#     ["pneumonia_1.jpg"],
#     ["pneumonia_2.jpg"],
# ]

# # Create Gradio Interface
# interface = gr.Interface(
#     fn=predict,
#     inputs=gr.Image(type="pil"),
#     outputs=gr.Label(),
#     title="Pneumonia Detection",
#     description="Upload a chest X-ray image or select an example below to classify as Normal or Pneumonia.",
#     examples=examples  # Add example images
# )

# # Launch the app
# if __name__ == "__main__":
#     interface.launch()

import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

# Load trained model
model = tf.keras.models.load_model("model.h5")

# Define class labels
class_labels = {0: "🟢 Normal", 1: "🔴 Pneumonia"}

# Preprocessing function (compatible with MobileNetV2)
def preprocess_image(img):
    img = img.resize((244, 244))  # Resize to match training size
    img = np.array(img)  # Convert to numpy array
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)  # Apply MobileNetV2 preprocessing
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Prediction function
def predict(img):
    img = preprocess_image(img)
    prediction = model.predict(img)[0]  # Get prediction
    class_index = np.argmax(prediction)  # Get highest probability class
    confidence = float(np.max(prediction))  # Get confidence score
    return {class_labels[class_index]: confidence}

# Example Images (Stored in the main directory)
examples = [
    ["normal_1.jpg"],
    ["normal_2.jpg"],
    ["pneumonia_1.jpg"],
    ["pneumonia_2.jpg"],
]

# Create Gradio Interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(),
    title="🩺 Pneumonia Detection AI",
    description="**Upload a chest X-ray image or select an example below to classify as Normal or Pneumonia.**\n\n⚡ **Powered by Deep Learning & MobileNetV2**",
    examples=examples,  # Add example images
    theme="default"
)

# Launch the app
if __name__ == "__main__":
    interface.launch()
